In [2]:
from warnings import filterwarnings
filterwarnings('ignore')
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model,Sequential
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization
from keras.layers import Activation,Dropout,Flatten,Dense,GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint,EarlyStopping,LearningRateScheduler
import numpy as np
seed = 9
np.random.seed(seed=seed)
tf.set_random_seed(seed=seed)
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

Using TensorFlow backend.


In [4]:
transformation_ratio = 0.05
momentum = 0.9
img_width,img_height = 416,416
batch_size = 32
nb_epoch= 100
learn_rate = 1e-4
nb_classes = 2

In [7]:
train_data_dir = 'Classification Dataset/train'
validation_data_dir = 'Classification Dataset/val'
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=15,
#                                    vertical_flip=True,
                                   fill_mode='reflect',
                                   data_format='channels_last',
                                   brightness_range=[0.5, 1.5],
                                   featurewise_center=True,
                                   featurewise_std_normalization=True)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                              target_size=(img_width, img_height),
                                                              batch_size=batch_size,
                                                              class_mode='categorical')

label_map = (train_generator.class_indices)
print(label_map)


Found 554 images belonging to 2 classes.
Found 138 images belonging to 2 classes.
{'phone_hand': 0, 'phone_head': 1}


In [8]:
def schedule(epoch, decay=0.9):
    return base_lr * decay**(epoch)

In [9]:
# base_lr = 1e-4
path_to_weights = "/custom_weights-{epoch:02d}-{val_loss:.2f}.h5"
callbacks_list = [
    ModelCheckpoint(path_to_weights,monitor = 'val_loss',verbose = 1,save_weights_only = False),
#     EarlyStopping(monitor = 'val_loss',patience = 5,verbose = 1)
#     LearningRateScheduler(schedule,verbose = 1)
]

In [14]:
input_shape = (416,416,3)
model = Sequential()

model.add(Conv2D(32, (3, 3), border_mode='same', input_shape=input_shape, activation='relu'))
model.add(Conv2D(32, (3, 3), border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), border_mode='same', activation='relu'))
model.add(Conv2D(64, (3, 3), border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), border_mode='same', activation='relu'))
model.add(Conv2D(128, (3, 3), border_mode='same', activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(256,(3, 3), border_mode='same', activation='relu'))
# model.add(Conv2D(256,(3, 3), border_mode='same', activation='relu'))
# model.add(Conv2D(256, (3, 3), border_mode='same', activation='relu'))
# # model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512,(3, 3), border_mode='same', activation='relu'))
model.add(Conv2D(512,(3, 3), border_mode='same', activation='relu'))
model.add(Conv2D(512, (3, 3), border_mode='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
# model.add(GlobalAveragePooling2D())
model.add(Dense(2))
model.add(Activation('sigmoid'))
    


In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 416, 416, 32)      896       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 416, 416, 32)      9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 208, 208, 32)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 208, 208, 64)      18496     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 208, 208, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 104, 104, 64)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 104, 104, 128)    

In [16]:
model.compile(loss='binary_crossentropy',
            optimizer=keras.optimizers.Adamax(lr=learn_rate, beta_1=0.9, beta_2=0.999),
            metrics=['accuracy'])

In [1]:
history = model.fit_generator(train_generator,
                             samples_per_epoch = train_generator.samples,
                             nb_epoch= nb_epoch,
                             validation_data = validation_generator,
                              nb_val_samples = validation_generator.samples,
                              callbacks = callbacks_list,
                              )

NameError: name 'model' is not defined

In [ ]:
import matplotlib.pyplot as plt

print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate_generator(validation_generator, validation_generator.samples/batch_size, workers=12)
# print(score)

In [60]:
scores = model.predict_generator(validation_generator, validation_generator.samples/batch_size, workers=12)

correct = 0
for i, n in enumerate(validation_generator.filenames):
    if n.startswith("Person_Talking_Phone") and scores[i][0] <= 0.5:
        correct += 1
    if n.startswith("Person_Using_Phone") and scores[i][0] > 0.5:
        correct += 1

print("Correct:", correct, " Total: ", len(validation_generator.filenames))
print("Loss: ", score[0], "Accuracy: ", score[1])

Correct: 76  Total:  158
Loss:  7.753007888793945 Accuracy:  0.5189873421493965


In [10]:
files = os.listdir("/home/u37306/Custom/Model_weights_416x416/")
files = [os.path.join("/home/u37306/Custom/Model_weights_416x416/", f) for f in files]
files.sort(key=lambda x: os.path.getmtime(x))
print(files[-1])
fname = files[-1].split('/')[-1]
epochs_done = int(fname.split("-")[1])

/home/u37306/Custom/Model_weights/custom_weights-60-0.73.h5


In [11]:
from keras import models
model = models.load_model(files[-1])
print(model)